In [96]:
import pandas as pd
import numpy as np
from os import listdir

In [199]:
IN_DIR = "/ebio/abt3_projects/TwinsUK_viromes_Shao_Pei/tmp/2_readsCleaning/7.coverageMatrices.sorted.bacteria/"
#OUT_DIR = ""

In [200]:
#Get all genomes accessions and lengths
genomesList = pd.read_csv(IN_DIR+"../0.bacterialGenomes/BacterialGenomes.length.txt",
                          names=["LENGTH"],index_col=0,sep='\t')

In [201]:
def logNormalization(covMatrix,referenceLengths):
    referenceLengthsSample = referenceLengths.loc[covMatrix.index]
    referenceLengthsSample.sort_values(by="LENGTH",inplace=True)
    covMatrix = covMatrix.loc[referenceLengthsSample.index]
    #Normalize abundance matrix and put nan values to bins greater than contig size
    binSize = 100000
    for index,row in covMatrix.iterrows():
        covMatrix.loc[index][(referenceLengths.loc[index]["LENGTH"]/np.int(binSize))+1:] = np.NAN
    covMatrixNormalized = np.log10(covMatrix.add(1,axis=1))
    return covMatrixNormalized

In [206]:
#Read bin coverage files and get a vector of the median coverage per sample
binCovMedian = {}
for f in listdir(IN_DIR):
    if "coverage.matrix.csv" in f:
        sample = f.split('.')[0]
        binCovMatrix = pd.read_csv(IN_DIR+f,header=0,index_col=0,sep='\t')
        binCovMatrixNormalized = logNormalization(binCovMatrix,genomesList)
        binCovMedian[sample] = binCovMatrixNormalized.median(axis=1)
#Tranform all per sample vector into one matrix
contaminantsMatrix = pd.DataFrame(binCovMedian)

In [208]:
#Build the matrix
#Build a matrix containing all the genomes and not only those with reads mapping to them. 
contaminantsMatrix = contaminantsMatrix.loc[genomesList.index.values]
#remove samples C,G and F
contaminantsMatrix = contaminantsMatrix[['Ab1', 'Ab2', 'B1', 'B2', 'Da1', 'Da2', 'E1', 'E2', 'H1', 'H2', 'I1', 'I2',
                                        'J1', 'J2', 'K1', 'K2', 'L1', 'L2', 'M1', 'M2', 'N1', 'N2', 'O1', 'O2', 
                                        'P1', 'P2', 'Q1', 'Q2', 'Ra1', 'Ra2', 'S1', 'S2', 'T1', 'T2', 'U1', 'U2', 
                                        'W1', 'W2', 'X1', 'X2', 'Y1', 'Y2']]

In [214]:
#Sort matrix by prevalence of the contaminants
#Make a presence/absence matrix 
contaminantsMatrixIO = contaminantsMatrix.copy()
contaminantsMatrixIO[contaminantsMatrixIO < 2] = 0
contaminantsMatrixIO[contaminantsMatrixIO >= 2] = 1
#Make a ~Prevalence vector
contaminantsPrevalence = contaminantsMatrixIO.sum(axis=1).sort_values(ascending=False)
#Sort the matrix according to the prevalence vector
contaminantsMatrix = contaminantsMatrix.loc[contaminantsPrevalence.index.values]
#Attach the prevalence info to the matrix
contaminantsMatrix["IO"] = contaminantsPrevalence

In [236]:
xInv = lambda x:10**x - 1

In [238]:
contaminantsMatrix.apply(xInv)

,Ab1,Ab2,B1,B2,Da1,Da2,E1,E2,H1,H2,...,T2,U1,U2,W1,W2,X1,X2,Y1,Y2,IO
CP009057.1,0.000000,346.000000,1998.000000,1221.000000,0.000000,1966.000000,6393.000000,4405.000000,2498.000000,621.000000,...,1753.000000,562.000000,4610.000000,0.0,1086.000000,15453.000000,296.000000,9818.000000,3439.000000,1.000000e+38
CP000139.1,0.000000,316.773504,2333.996574,1257.130359,0.000000,2429.195054,8284.938209,4718.989407,2832.086656,403.462606,...,2153.420572,744.945708,5731.767656,0.0,1662.490908,18119.591050,399.988778,11535.941709,4214.374716,1.000000e+37
CP013020.1,0.000000,204.450724,2196.868968,1261.993666,0.000000,2685.629487,9406.882599,4760.892481,2906.982806,458.510609,...,1831.480286,769.999351,6199.447564,0.0,1579.496125,18136.452522,377.967017,11334.497519,4236.811228,1.000000e+37
FP929051.1,642.000000,1176.000000,25077.000000,165421.000000,107457.000000,5039.000000,36552.000000,767.000000,14876.000000,22753.000000,...,126609.000000,0.000000,0.000000,0.0,40318.000000,6538.000000,4822.000000,0.000000,6165.000000,1.000000e+37
CP011531.1,0.000000,246.741801,1948.333219,1502.943815,0.000000,2140.306610,6894.497807,4468.443366,2690.988113,681.498352,...,1662.494815,577.136662,4770.496830,0.0,1151.053384,15857.519477,396.979899,9515.465836,3417.999415,1.000000e+36
FP929046.1,820.000000,425.000000,10367.000000,1513.000000,9324.000000,635.000000,17139.000000,0.000000,1098.000000,4018.000000,...,474.000000,671.000000,0.000000,251.0,4534.000000,983.000000,0.000000,0.000000,1814.000000,1.000000e+34
FP929045.1,329.136335,7.660254,2779.352675,377.893125,34317.107815,593.191888,181439.336144,282.955983,800.511073,4019.537278,...,1647.899633,628.142273,0.000000,0.0,12142.810893,1739.232743,5.782330,0.000000,706.283536,1.000000e+31
FP929033.1,2684.884584,0.000000,0.000000,0.000000,3128.466408,373.966665,116.371206,966.453358,375.952252,0.000000,...,0.000000,164.987951,2548.999216,NaN,0.000000,828.839744,0.000000,1350.178005,151.262930,1.000000e+24
CP002544.1,10932.607822,0.000000,0.000000,0.000000,4194.924451,334.343108,2654.995294,377.868051,121.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.000000,3101.995972,0.000000,417.855584,610.189005,1.000000e+21
CP015401.1,3196.000000,0.000000,0.000000,0.000000,2563.000000,302.000000,156.000000,682.000000,403.000000,0.000000,...,0.000000,0.000000,1380.000000,0.0,0.000000,791.000000,0.000000,895.000000,0.000000,1.000000e+21


In [237]:
for x in contaminantsMatrix:
    contaminantsMatrix[x].apply(xInv)

Ab1
Ab2
B1
B2
Da1
Da2
E1
E2
H1
H2
I1
I2
J1
J2
K1
K2
L1
L2
M1
M2
N1
N2
O1
O2
P1
P2
Q1
Q2
Ra1
Ra2
S1
S2
T1
T2
U1
U2
W1
W2
X1
X2
Y1
Y2
IO


In [232]:
np.exp(contaminantsMatrix).head()

,Ab1,Ab2,B1,B2,Da1,Da2,E1,E2,H1,H2,...,T2,U1,U2,W1,W2,X1,X2,Y1,Y2,IO
CP009057.1,1.000000,12.683849,27.134685,21.912806,1.000000,26.945178,44.959974,38.246211,29.897243,16.342848,...,25.636820,15.650587,39.009107,1.0,20.826569,65.959499,11.855080,54.166750,34.348503,3.185593e+16
CP000139.1,1.000000,12.208320,29.028736,22.191862,1.000000,29.536924,50.316934,39.406904,31.571659,13.556666,...,28.031545,17.684830,42.878234,1.0,25.053673,70.680686,13.505976,58.095645,37.518577,1.171914e+16
CP013020.1,1.000000,10.101740,28.275672,22.229076,1.000000,30.852191,53.169870,39.558460,31.931463,14.329145,...,26.128830,17.940380,44.363765,1.0,24.502948,70.709242,13.178694,57.652902,37.605172,1.171914e+16
FP929051.1,16.580229,21.558636,81.393295,184.674214,153.122576,40.545733,95.863282,17.910036,64.878440,78.027221,...,164.427215,1.000000,1.000000,1.0,100.033962,45.399965,39.778130,1.000000,44.256556,1.171914e+16
CP011531.1,1.000000,10.957228,26.839806,23.980275,1.000000,27.957314,46.458787,38.484418,30.878901,17.015107,...,25.053698,15.831957,39.593091,1.0,21.358987,66.703848,13.461869,53.435520,34.257278,4.311232e+15


In [218]:
contaminantsMatrix.to_csv("contaminantsMedianBin.allgenomes.csv",header=True,index=True,sep='\t')

In [219]:
#Get info of the genomes
from Bio import Entrez
Entrez.email = "jmoreno@tuebingen.mpg.de"

In [ ]:
genomes = {}
i = 0
for genome in contaminantsMatrix.index.values:
    handle = Entrez.efetch(db="nucleotide", id=genome, rettype="gb", retmode="xml")
    record = Entrez.read(handle)
    genomes[genome] = [record[0]["GBSeq_organism"],record[0]["GBSeq_length"],record[0]["GBSeq_taxonomy"]]
    i += 1
    if i%1000 == 0:
        print i

In [223]:
len(genomes)

883